In [1]:
!pip install firebase

In [2]:
from firebase import firebase
from IPython.display import HTML, display
from google.colab import output
from nltk.stem import PorterStemmer
import re
import requests
from bs4 import BeautifulSoup
import nltk
import json

#Connect to firebase
FBconn = firebase.FirebaseApplication("https://cloudproject-1bbde-default-rtdb.firebaseio.com/", None)

In [3]:
def fetch_page(url):
  nltk.download('punkt')
  links = []
  response = requests.get(url)
  if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    anchorTags = soup.find_all('a')

    for anchorTag in anchorTags:
      link = anchorTag.get('href')
      if link:
        links.append(link)
    return soup, links
  else:
    return None



def extract_data_from_url(soup):
  # Extract text content from the HTML
  text_content = soup.get_text()

  # Tokenize text into sentences using NLTK
  sentences = nltk.sent_tokenize(text_content)

  # Remove newline characters from each sentence and store in the 'data' list
  data = [sentence.replace('\n', '') for sentence in sentences]
  return data


def create_json_object(json_object, url, data):
  json_object[url] = data
  # print(json_object)

def create_index(url, jsonData):
  print("THIS IS FROM CREATE_INDEX", url)
  soup, links = fetch_page(url)
  print("THIS IS THE LINKS FROM CREATE_INDEX")

  for link in links:
      print(link)
      data = extract_data_from_url(soup)
      create_json_object(jsonData, link, data)

  return jsonData


def remove_stop_words(query):
  #Remove all the stop words from the query
  stop_words = {'a', 'an', 'and', 'are', 'as', 'at', 'be', 'but', 'by', 'for', 'if', 'in', 'into', 'is', 'it', 'no', 'not', 'of', 'on', 'or', 'such', 'that', 'the', 'their', 'then',
                'there', 'these', 'they', 'this', 'to', 'was', 'will', 'with'}
  words = query.split()
  filtered_words = [word for word in words if word.lower() not in stop_words]

  result =  ' '.join(filtered_words)
  return result


def clean_special_characters(text):
    # Replace non-ASCII characters with an empty string
    cleaned_text = re.sub(r'[^\x00-\x7F]+', '', text)
    return cleaned_text



In [4]:
_, links = fetch_page('https://www.ibm.com/cloud')

for link in links:
  _, new_links = fetch_page(link)
  print(new_links)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['https://www.ibm.com/account/reg/signup?formid=urx-52652', 'https://www.ibm.com/account/reg/signup?formid=urx-52651']
['https://www.ibm.com/account/reg/signup?formid=urx-49524', 'https://www.ibm.com/partnerplus']


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['https://www.ibm.com/downloads/cas/QE7BZ7A5', 'https://www.ibm.com/blog/announcement/ibm-cloud-vmware-as-a-service-introduces-multitenant-as-a-new-cost-efficient-consumption-model/']


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[]
[]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
_, new_links = fetch_page('https://www.ibm.com/blog/announcement/ibm-cloud-vmware-as-a-service-introduces-multitenant-as-a-new-cost-efficient-consumption-model/')
print(new_links)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['https://www.ibm.com/blog/category/cloud/', 'https://www.ibm.com/blog/author/sami-kuronen/', 'https://www.ibm.com/blog/author/bryan-buckland/', 'https://www.ibm.com/blog/author/phil-fritz/', 'https://twitter.com/intent/tweet?text=IBM Cloud VMware as a Service introduces multitenant as a new, cost-efficient consumption model&url=https://www.ibm.com/blog/announcement/ibm-cloud-vmware-as-a-service-introduces-multitenant-as-a-new-cost-efficient-consumption-model/', 'https://www.linkedin.com/shareArticle?url=https://www.ibm.com/blog/announcement/ibm-cloud-vmware-as-a-service-introduces-multitenant-as-a-new-cost-efficient-consumption-model/', 'https://www.facebook.com/sharer/sharer.php?u=https://www.ibm.com/blog/announcement/ibm-cloud-vmware-as-a-service-introduces-multitenant-as-a-new-cost-efficient-consumption-model/', 'https://www.ibm.com/blog/announcement/ibm-cloud-vmware-as-a-service-introduces-multitenant-as-a-new-cost-efficient-consumption-model/', 'https://www.ibm.com/products/vmwar

In [6]:
pip install --upgrade certifi

In [7]:
from collections import deque
from urllib.parse import urljoin
from notebook.services.config import ConfigManager
import logging
import chardet
import time

cm = ConfigManager()

nltk.download('punkt')

cm.update('NotebookApp', {'iopub_data_rate_limit': 1000000000000})

# Configure logging
logging.basicConfig(level=logging.INFO)

def is_valid_url(url):
    return url.startswith(('http://', 'https://')) and not url.startswith('javascript:')

# def fetch_and_process_page(url):
#     try:
#         response = requests.get(url)
#         response.raise_for_status()
#         encoding = chardet.detect(response.content)['encoding'] or 'utf-8'
#         cleaned_html = response.content.decode(encoding)
#         soup = BeautifulSoup(cleaned_html, 'lxml')
#         return soup
#     except requests.exceptions.RequestException as e:
#         logging.error(f"Error fetching {url}: {e}")
#     except Exception as e:
#         logging.error(f"Error processing {url}: {e}")
#     return None

def fetch_page_iterative(url, depth, visited=None):
    if visited is None:
        visited = set()

    # Early return if the URL has already been visited
    if url in visited or not is_valid_url(url):
        return []

    visited.add(url)  # Mark the current URL as visited

    if depth == 0:
        return []

    links = []
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            anchorTags = soup.find_all('a')

            for anchorTag in anchorTags:
                link = anchorTag.get('href', '')
                if is_valid_url(link):
                    links.append(link)
                    # Extend the list with links from deeper levels, if valid and not yet visited
                    child_links = fetch_page_iterative(link, depth - 1, visited)
                    links.extend(child_links)
    except Exception as e:
        print(f"Error fetching {url}: {e}")

    # Return the list of links, deduplicated by converting it to a set and back to a list
    return list(set(links))

def save_to_json_file(data, file_path):
    with open(file_path, 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, ensure_ascii=False)

# Remember to define or adjust the `extract_data_from_url` function according to your needs.


# Example usage
# url_to_start = "https://www.ibm.com/cloud"
# max_recursive_depth = 4
# discovered_links_data = fetch_page_iterative(url_to_start, max_depth=max_recursive_depth)

# Convert results to JSON and save to a file
# json_file_path = "discovered_links_data.json"
# save_to_json_file(discovered_links_data, json_file_path)

# Upload the JSON file to Firebase
# try:
    # with open(json_file_path, 'r', encoding='utf-8') as json_file:
        # json_data = json.load(json_file)
        # print(json_file)
        # json_string = json.dumps(json_data, ensure_ascii=False)
        # result = FBconn.post('/results/', json_string)
    # with open(json_file_path, 'r', encoding='utf-8') as json_file:
    #     json_data = json.load(json_file)
    #     # print(json_file)
    #     json_string = json.dumps(json_data, ensure_ascii=False)
    #     result = FBconn.post('/results/', json_string)

    # print("Data successfully posted to Firebase.")
# except Exception as e:
    # print(f"Error posting data to Firebase: {e}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import re

# Initialize Firebase Admin SDK
cred = credentials.Certificate('cloudproject-1bbde-firebase-adminsdk-849hb-47666b58c8.json') # Update the path to your downloaded JSON file
# firebase_admin.initialize_app(cred)

# Get a Firestore client
db = firestore.client()

# # Path to your Firebase Admin SDK JSON file
# cred_path = 'path/to/your/firebase-adminsdk.json'

# # Initialize the Firebase Admin SDK if it hasn't been initialized yet
# if not firebase_admin._apps:
#     cred = credentials.Certificate(cred_path)
#     firebase_admin.initialize_app(cred, {
#         'databaseURL': 'https://test-1eb65-default-rtdb.firebaseio.com/' # Replace with your Realtime Database URL
#     })

def build_index(urls):
    """
    Builds an inverted index from a list of URLs, treating uppercase and lowercase words as the same and excluding non-word characters.
    :param urls: A list of URLs to index.
    :return: An inverted index where keys are terms (in lowercase) and values are lists of URLs where the term appears, excluding punctuation.
    """
    inverted_index = {}

    for url in urls:
        try:
            soup, links = fetch_page(url)
            if soup is not None:
                data = extract_data_from_url(soup)
                for sentence in data:
                    # Optional: clean the text and remove stop words
                    sentence = clean_special_characters(sentence)
                    filtered_sentence = remove_stop_words(sentence)
                    # Convert to lowercase to ensure uniformity
                    filtered_sentence = filtered_sentence.lower()
                    # Use regular expressions to find all words in the sentence, excluding numbers
                    words = re.findall(r'\b(?![0-9]+\b)\w+\b', filtered_sentence)
                    for word in words:
                        if word not in inverted_index:
                            inverted_index[word] = [url]
                        elif url not in inverted_index[word]:
                            inverted_index[word].append(url)
        except Exception as e:
            print(f"Error processing {url}: {e}")

    return inverted_index

def upload_index_to_firestore(inverted_index):
    """
    Uploads the inverted index to Firestore.
    :param inverted_index: The inverted index to upload.
    """
    # Reference a collection. If it doesn't exist, it will be created.
    index_collection = db.collection('inverted_index')

    for term, urls in inverted_index.items():
        # Each term will be a document in the 'inverted_index' collection
        # The URLs where the term appears will be stored in the 'urls' field
        index_collection.document(term).set({'urls': urls})

    print("Inverted index uploaded to Firestore.")

# After building your inverted index


# urls = ['https://www.ibm.com/products/security-and-compliance-center', 'https://www.ibm.com/products/satellite', 'https://www.ibm.com/products/vmware', 'https://www.ibm.com/cloud/financial-services', 'https://www.ibm.com/cloud/solutions']
# urls = fetch_page_iterative('https://www.ibm.com/cloud', 10)
# print(urls)
# inverted_index = build_index(urls)  # Make sure to replace `your_urls_list` with your actual list of URLs
# upload_index_to_firestore(inverted_index)
# upload_index_to_realtime_database(build_index(urls))
# print(build_index(urls))

base_url = 'https://www.ibm.com/cloud'
depth = 1  # For example, to stop after 2 iterations

# Fetch links up to the specified depth
urls = fetch_page_iterative(base_url, depth)
print(f"Fetched {len(urls)} URLs at depth {depth}")
print(urls)
# Build the index from the fetched URLs
inverted_index = build_index(urls)
# print("Inverted index built.")
print(inverted_index)

# Upload the index to Firestore
upload_index_to_firestore(inverted_index)
# print("Inverted index uploaded to Firestore.")

Fetched 5 URLs at depth 1
['https://www.ibm.com/cloud/solutions', 'https://www.ibm.com/cloud/financial-services', 'https://www.ibm.com/products/security-and-compliance-center', 'https://www.ibm.com/products/satellite', 'https://www.ibm.com/products/vmware']


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


{'cloud': ['https://www.ibm.com/cloud/solutions', 'https://www.ibm.com/cloud/financial-services', 'https://www.ibm.com/products/security-and-compliance-center', 'https://www.ibm.com/products/satellite', 'https://www.ibm.com/products/vmware'], 'solutions': ['https://www.ibm.com/cloud/solutions', 'https://www.ibm.com/cloud/financial-services', 'https://www.ibm.com/products/security-and-compliance-center', 'https://www.ibm.com/products/satellite', 'https://www.ibm.com/products/vmware'], 'ibm': ['https://www.ibm.com/cloud/solutions', 'https://www.ibm.com/cloud/financial-services', 'https://www.ibm.com/products/security-and-compliance-center', 'https://www.ibm.com/products/satellite', 'https://www.ibm.com/products/vmware'], 'run': ['https://www.ibm.com/cloud/solutions', 'https://www.ibm.com/products/satellite', 'https://www.ibm.com/products/vmware'], 'enterprise': ['https://www.ibm.com/cloud/solutions', 'https://www.ibm.com/cloud/financial-services', 'https://www.ibm.com/products/satellite'

In [15]:
print(inverted_index)

{'cloud': ['https://www.ibm.com/cloud/solutions', 'https://www.ibm.com/cloud/financial-services', 'https://www.ibm.com/products/security-and-compliance-center', 'https://www.ibm.com/products/satellite', 'https://www.ibm.com/products/vmware'], 'solutions': ['https://www.ibm.com/cloud/solutions', 'https://www.ibm.com/cloud/financial-services', 'https://www.ibm.com/products/security-and-compliance-center', 'https://www.ibm.com/products/satellite', 'https://www.ibm.com/products/vmware'], 'ibm': ['https://www.ibm.com/cloud/solutions', 'https://www.ibm.com/cloud/financial-services', 'https://www.ibm.com/products/security-and-compliance-center', 'https://www.ibm.com/products/satellite', 'https://www.ibm.com/products/vmware'], 'run': ['https://www.ibm.com/cloud/solutions', 'https://www.ibm.com/products/satellite', 'https://www.ibm.com/products/vmware'], 'enterprise': ['https://www.ibm.com/cloud/solutions', 'https://www.ibm.com/cloud/financial-services', 'https://www.ibm.com/products/satellite'

In [13]:
print(f"Length of inverted index: {len(inverted_index)}")

Length of inverted index: 1185


In [14]:
total_word_occurrences = sum(len(urls) for urls in inverted_index.values())
print(f"Total number of word occurrences (URLs listed): {total_word_occurrences}")


Total number of word occurrences (URLs listed): 1828
